In [18]:
import pandas as pd
from tqdm import tqdm
import os
import json
from glom import glom

In [19]:
df = pd.read_json('../datasets/merged_data/champion.json')
df= df.T

df['key'] = pd.to_numeric(df['key'], errors='ignore')
df.sort_values("key" , ascending=True,inplace=True)
df.head(5)

,version,id,key,name,title,blurb,info,image,tags,partype,stats
Annie,12.12.1,Annie,1,Annie,the Dark Child,"Dangerous, yet disarmingly precocious, Annie i...","{'attack': 2, 'defense': 3, 'magic': 10, 'diff...","{'full': 'Annie.png', 'sprite': 'champion0.png...",[Mage],Mana,"{'hp': 594, 'hpperlevel': 102, 'mp': 418, 'mpp..."
Olaf,12.12.1,Olaf,2,Olaf,the Berserker,"An unstoppable force of destruction, the axe-w...","{'attack': 9, 'defense': 5, 'magic': 3, 'diffi...","{'full': 'Olaf.png', 'sprite': 'champion2.png'...","[Fighter, Tank]",Mana,"{'hp': 645, 'hpperlevel': 119, 'mp': 316, 'mpp..."
Galio,12.12.1,Galio,3,Galio,the Colossus,"Outside the gleaming city of Demacia, the ston...","{'attack': 1, 'defense': 10, 'magic': 6, 'diff...","{'full': 'Galio.png', 'sprite': 'champion1.png...","[Tank, Mage]",Mana,"{'hp': 632, 'hpperlevel': 126, 'mp': 500, 'mpp..."
TwistedFate,12.12.1,TwistedFate,4,Twisted Fate,the Card Master,Twisted Fate is an infamous cardsharp and swin...,"{'attack': 6, 'defense': 2, 'magic': 6, 'diffi...","{'full': 'TwistedFate.png', 'sprite': 'champio...",[Mage],Mana,"{'hp': 604, 'hpperlevel': 108, 'mp': 333, 'mpp..."
XinZhao,12.12.1,XinZhao,5,Xin Zhao,the Seneschal of Demacia,Xin Zhao is a resolute warrior loyal to the ru...,"{'attack': 8, 'defense': 6, 'magic': 3, 'diffi...","{'full': 'XinZhao.png', 'sprite': 'champion4.p...","[Fighter, Assassin]",Mana,"{'hp': 640, 'hpperlevel': 106, 'mp': 274, 'mpp..."


In [20]:
df_champion = df[['name', 'key']]
df_champion.columns=['name', 'id']
df_champion


,name,id
Annie,Annie,1
Olaf,Olaf,2
Galio,Galio,3
TwistedFate,Twisted Fate,4
XinZhao,Xin Zhao,5
...,...,...
Yone,Yone,777
Sett,Sett,875
Lillia,Lillia,876
Gwen,Gwen,887


In [21]:
df_champion.to_csv( "champion.csv", index= False)

In [22]:
df_champion['cj'] = 0
df_champion2 = df_champion.copy()

/var/folders/9r/4qn8djgj1_g7m5m_fylb0s7w0000gn/T/ipykernel_14223/4292177480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_champion['cj'] = 0


In [23]:
combo = df_champion.merge(df_champion2, on='cj', how='outer')

combo

,name_x,id_x,cj,name_y,id_y
0,Annie,1,0,Annie,1
1,Annie,1,0,Olaf,2
2,Annie,1,0,Galio,3
3,Annie,1,0,Twisted Fate,4
4,Annie,1,0,Xin Zhao,5
...,...,...,...,...,...
25595,Renata Glasc,888,0,Yone,777
25596,Renata Glasc,888,0,Sett,875
25597,Renata Glasc,888,0,Lillia,876
25598,Renata Glasc,888,0,Gwen,887


In [24]:
combo.drop(['cj'], axis=1, inplace=True)
combo = combo.loc[combo['id_y'] > combo['id_x']]
combo.columns = ['champion1','id1','champion2','id2']
combo['combo'] = 0

combo

/var/folders/9r/4qn8djgj1_g7m5m_fylb0s7w0000gn/T/ipykernel_14223/649456937.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combo['combo'] = 0


,champion1,id1,champion2,id2,combo
1,Annie,1,Olaf,2,0
2,Annie,1,Galio,3,0
3,Annie,1,Twisted Fate,4,0
4,Annie,1,Xin Zhao,5,0
5,Annie,1,Urgot,6,0
...,...,...,...,...,...
25118,Sett,875,Gwen,887,0
25119,Sett,875,Renata Glasc,888,0
25278,Lillia,876,Gwen,887,0
25279,Lillia,876,Renata Glasc,888,0


In [25]:
combo.to_csv( "combo.csv", index= False)